### United States Real Estate Analysis ###

Please Note: Our primary csv for real estate data contained more than 2M datapoints and was significantly larger than GitHub allowed (170.57 MB). As such, we trimmed down our dataset logically with the following code prior to upload:

-Removed unnecessay columns (Brokered By & Street) by only including columns of interest: dataframe = dataframe[["status", "price", "bed", "bath", "acre_lot", "city", "state", "zip_code", "house_size", "prev_sold_date"]]

-Removed U.S. Territories (Puerto Rico & Virgin Islands) to only include states within our analysis: dataframe = dataframe.loc[(dataframe["state"] != "Puerto Rico") & (dataframe["state"] != "Virgin Islands")]

In [1]:
# Import appropriate dependencies
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
from scipy.stats import linregress
from scipy.stats import normaltest
import time

# Establish proper filepath to csv file
filepath = "Resources/real_estate.csv"

# Read csv file into initial dataframe and display
initial_dataset_df = pd.read_csv(filepath)
initial_dataset_df

,status,price,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date
0,for_sale,525000,3,3,0.45,Agawam,Massachusetts,1001,2314,6/25/14
1,for_sale,289900,3,2,0.36,Agawam,Massachusetts,1001,1276,10/12/12
2,for_sale,384900,3,2,0.46,Agawam,Massachusetts,1001,1476,11/20/86
3,for_sale,199999,3,2,1.76,Agawam,Massachusetts,1001,1968,9/19/08
4,for_sale,419000,4,2,2.00,Pelham,Massachusetts,1002,1607,7/25/05
...,...,...,...,...,...,...,...,...,...,...
1048570,sold,520000,3,3,0.09,Newberg,Oregon,97132,1630,2/18/22
1048571,sold,635000,5,3,0.14,Beaverton,Oregon,97007,2618,2/18/22
1048572,sold,749000,4,2,0.20,Wilsonville,Oregon,97070,2492,2/18/22
1048573,sold,485000,3,2,0.13,Newberg,Oregon,97132,1364,2/18/22


In [2]:
# First, we should locate if there are any duplicate entries within our dataset
initial_dataset_df.duplicated().sum()

1216

In [3]:
# Drop the 1,216 duplicate values from our dataframe
initial_dataset_df.drop_duplicates(inplace=True)

In [4]:
# Confirm new count of rows minus the removed duplicate datapoints
initial_dataset_df.count()

status            1047359
price             1047359
bed               1047359
bath              1047359
acre_lot          1047359
city              1047359
state             1047359
zip_code          1047359
house_size        1047359
prev_sold_date    1047359
dtype: int64

In [7]:
# Coverting date format to datetime within pandas
initial_dataset_df['prev_sold_date'] = pd.to_datetime(initial_dataset_df['prev_sold_date'], format='%m/%d/%y')

In [8]:
# Display updated dataframe
initial_dataset_df

,status,price,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date
0,for_sale,525000,3,3,0.45,Agawam,Massachusetts,1001,2314,2014-06-25
1,for_sale,289900,3,2,0.36,Agawam,Massachusetts,1001,1276,2012-10-12
2,for_sale,384900,3,2,0.46,Agawam,Massachusetts,1001,1476,1986-11-20
3,for_sale,199999,3,2,1.76,Agawam,Massachusetts,1001,1968,2008-09-19
4,for_sale,419000,4,2,2.00,Pelham,Massachusetts,1002,1607,2005-07-25
...,...,...,...,...,...,...,...,...,...,...
1048570,sold,520000,3,3,0.09,Newberg,Oregon,97132,1630,2022-02-18
1048571,sold,635000,5,3,0.14,Beaverton,Oregon,97007,2618,2022-02-18
1048572,sold,749000,4,2,0.20,Wilsonville,Oregon,97070,2492,2022-02-18
1048573,sold,485000,3,2,0.13,Newberg,Oregon,97132,1364,2022-02-18


In [9]:
# Filtering number of bedroom to less than or equal to 5 & dates after 2015
filtered_df = initial_dataset_df.loc[(initial_dataset_df['bed'] <= 5) & (initial_dataset_df['prev_sold_date'] > "2015-01-01")]

In [10]:
# Splitting dataset into three distinct individual states for analysis (East US / Central US/ West US)
pa_df = initial_dataset_df.loc[(initial_dataset_df['state'] == "Pennsylvania")]
wi_df = initial_dataset_df.loc[(initial_dataset_df['state'] == "Wisconsin")]
wa_df = initial_dataset_df.loc[(initial_dataset_df['state'] == "Washington")]

In [11]:
# Display Pennsylvania dataframe
pa_df

,status,price,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date
18640,for_sale,279900,3,3,1.14,Lower Mt Bethel Township,Pennsylvania,18013,1865,2017-08-01
18641,for_sale,129900,3,1,0.56,East Stroudsburg,Pennsylvania,18301,1050,1970-11-10
18643,for_sale,189900,2,3,0.03,Stroudsburg,Pennsylvania,18360,1230,2005-04-01
18645,for_sale,259900,3,2,0.13,East Stroudsburg,Pennsylvania,18301,1306,2021-09-03
18646,for_sale,135900,3,1,0.09,Bangor,Pennsylvania,18013,904,2019-03-26
...,...,...,...,...,...,...,...,...,...,...
727642,sold,210000,3,2,0.50,Beaver Falls,Pennsylvania,15010,2016,2022-04-22
727644,sold,219000,7,4,0.16,New Brighton,Pennsylvania,15066,5600,2022-04-21
727784,sold,169900,2,1,1.09,New Castle,Pennsylvania,16105,1165,2021-11-09
727813,sold,255000,4,3,0.46,New Castle,Pennsylvania,16105,2450,2022-01-20


In [12]:
# Display Washington dataframe
wa_df

,status,price,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date
365274,for_sale,282000,4,2,0.32,Clarkston,Washington,99403,1628,2007-01-31
365275,for_sale,490000,3,2,0.42,Clarkston,Washington,99403,1789,2006-04-18
365278,for_sale,479000,4,4,0.24,Clarkston,Washington,99403,2652,2007-06-04
365279,for_sale,550000,4,3,0.27,Clarkston,Washington,99403,2510,2015-04-17
365281,for_sale,649000,4,2,0.27,Asotin,Washington,99402,1623,2021-01-07
...,...,...,...,...,...,...,...,...,...,...
1046588,sold,60000,2,1,0.26,Wishram,Washington,98673,1160,2022-01-13
1046653,sold,314370,3,2,5.07,Goldendale,Washington,98620,1763,2022-04-27
1046701,sold,410000,3,2,0.55,Dallesport,Washington,98617,1809,2021-11-22
1046703,sold,250000,2,2,45.00,Goldendale,Washington,98620,924,2021-11-19


In [13]:
# Display Wisconsin dataframe
wi_df

,status,price,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date
238260,for_sale,179900,2,1,2.00,Pembine,Wisconsin,54156,1248,2020-08-20
238305,for_sale,239900,3,2,26.37,Pembine,Wisconsin,54156,1630,2017-04-13
238307,for_sale,179900,3,2,1.10,Pembine,Wisconsin,54156,1824,2004-07-29
238309,for_sale,320000,3,2,36.99,Wausaukee,Wisconsin,54177,1626,2003-04-08
238310,for_sale,339000,3,2,2.54,Florence,Wisconsin,54121,1684,2017-06-12
...,...,...,...,...,...,...,...,...,...,...
785402,sold,79900,3,2,0.26,Janesville,Wisconsin,53548,1588,2022-03-25
785403,sold,115000,3,2,0.14,Beloit,Wisconsin,53511,1006,2022-03-25
785404,sold,262500,2,2,0.26,Janesville,Wisconsin,53545,1215,2022-04-08
785405,sold,149900,2,2,0.17,Beloit,Wisconsin,53511,1092,2022-04-08
